In [47]:
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import time 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.regularizers import l2
import os

In [3]:
def showHsv(img):    
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    plt.imshow(img)
    plt.show()

In [65]:
img_in = cv2.imread('pic.jpg')
# maybe should switch to LAB colorspace?
hsv = cv2.cvtColor(img_in, cv2.COLOR_BGR2HSV)
print(hsv.shape)

h,s,v = cv2.split(hsv)
#print('s',s.shape)
s = (np.random.random(s.shape) * 255).astype('uint8')
h = (np.random.random(s.shape) * 255).astype('uint8')
#s = np.maximum(0, v)
#print(s)
hsv = cv2.merge([h,s,v])
print('image with its value untouched, but saturation and hue randomized:')
#showHsv(hsv)





(1008, 1600, 3)
image with its value untouched, but saturation and hue randomized:


In [11]:
for i in range(3):
    break
    pic = hsv[:,:,i]
    plt.imshow(pic, cmap='gray')
    plt.show()

In [13]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


In [14]:
n = 10000
X = np.arange(n).reshape(n,1)
X = np.hstack((np.ones(n).reshape(n,1), X))
print(X.shape)
y = np.random.randn(n)*50 + X[:, 1]
#y = y.reshape(n,1)
print(y.shape, X[:,1].shape)
#print(y)
#plt.plot(X[:,1], y)
yval = y
X = np.hstack((X, yval[:,None]))
# labels: 1 if above line, 0  below
y = y > X[:,1]
y = y.reshape(n,1)
y = np.hstack((y, ~y )).astype('uint8')
print(y.shape)
colors = ['red' if i  else 'green' for i in y[:,0]]
#plt.scatter(X[:,1], yval, color=colors)
#plt.show()
#print(y)
#print(X)


(10000, 2)
(10000,) (10000,)
(10000, 2)


In [39]:
def createTestModel(layers):
    model = Sequential()
    #print(X.shape[1])
    model.add(Dense(layers[0], input_dim=X.shape[1], kernel_regularizer=l2(0.001)) ) 
    model.add(Activation('relu'))
    for l in layers[1:]:
        model.add(Dense(l, kernel_regularizer=l2(0.001)))
        model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                 metrics=['accuracy'])
    return model

In [85]:
def loadImgHsv(path):
    img_in = cv2.imread(path)
    hsv = cv2.cvtColor(img_in, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    X = v
    Y = np.dstack((h,s))
    #print("loadX.shape", X.shape, 'loadY.shape', Y.shape)
    return X, Y

# celebA dataset from https://drive.google.com/drive/folders/0B7EVK8r0v71pTUZsaXdaSnZBZzg
# 202,599 images of size 178x218

# data generator for feeding into memory parts of dataset
class DataGenerator(tf.keras.utils.Sequence):
    'data generator for feeding into memory parts of dataset'
    def __init__(self, list_IDs, batch_size=32, dim=(178,218), in_channels=1,
                 out_channels=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, Y = self.__data_generation(list_IDs_temp)

        return X, Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        # target 2 channels
        Y = np.empty((self.batch_size, *self.dim, self.out_channels))
        print(X.shape, Y.shape)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i], Y[i] = loadImgHsv('data/img_celeba/' + ID)

        return X, Y

In [95]:
params = {'dim': (218, 178),
          'batch_size': 32,
          'in_channels': 1,
          'out_channels': 2,
          'shuffle': True}

train_IDs = os.listdir('data/img_celeba/')
splitAt = int(len(train_IDs)*0.1)
train_IDs = train_IDs[:splitAt]
print(train_IDs[:10], len(train_IDs))

training_generator = DataGenerator(train_IDs, **params)
633*32

['000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005.jpg', '000006.jpg', '000007.jpg', '000008.jpg', '000009.jpg', '000010.jpg'] 20259


20256

In [44]:
currTime = time.strftime("%d-%m-%Y-%H-%M-%S", time.localtime())
layers = [5,12,5]
NAME = f"test-tensorboard-{ currTime }-layers-" + "x".join(str(l) for l in layers)
print(NAME)

model = createTestModel(layers)
tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2, callbacks=[tensorboard])

test-tensorboard-17-03-2019-10-56-17-layers-5x12x5
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 100us/step - loss: 0.1245 - acc: 0.9625 - val_loss: 0.2851 - val_acc: 0.8965
Epoch 2/10
8000/8000 [==============================] - 1s 100us/step - loss: 0.1904 - acc: 0.9518 - val_loss: 0.0405 - val_acc: 0.9962
Epoch 3/10
8000/8000 [==============================] - 1s 99us/step - loss: 0.1421 - acc: 0.9567 - val_loss: 0.0860 - val_acc: 0.9635
Epoch 4/10
8000/8000 [==============================] - 1s 97us/step - loss: 0.1966 - acc: 0.9441 - val_loss: 0.2634 - val_acc: 0.8998
Epoch 5/10
8000/8000 [==============================] - 1s 99us/step - loss: 0.4558 - acc: 0.9181 - val_loss: 0.1130 - val_acc: 0.9485
Epoch 6/10
8000/8000 [==============================] - 1s 98us/step - loss: 0.2290 - acc: 0.9447 - val_loss: 4.8313 - val_acc: 0.5962
Epoch 7/10
8000/8000 [==============================] - 1s 99us/step - loss: 0.2181 - acc

In [ ]:
#model.get_config()

In [83]:
pred = model.predict_classes(X).reshape((-1,))
pred = 1 - pred
#print(pred)
evaluation = model.evaluate(X, y)
print(f'loss:{evaluation[0]}\naccuracy:{evaluation[1]}')
labels = y[:,0]
#print(labels.shape, labels)
incorrects = np.nonzero(np.not_equal(pred, labels) )
print('num incorrect', len(incorrects[0]))
#print('incorr:',incorrects)
print('\nwrong pred:', pred[incorrects])


1000/1000 [==============================] - 0s 35us/step
loss:0.12557350927591324
accuracy:0.951
num incorrect 48

wrong pred: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1]
